In [331]:
import json
import re

In [332]:
def read_output_file(path, reverse=False):
    # path = "../experiments/somos/ab_testing/diff15_prompt1.txt"
    outputs = []
    with open(path, "r") as f:
        for line in f:
            x = json.loads(line)
            outputs.append(x)
    
    count_all, count_correct = 0, 0
    predictions = []
    for output in outputs:
        if not reverse:
            audio_a, audio_b = output['data']
        else:
            audio_b, audio_a = output['data']
        
        judge_text = output['response']

        # typo lol
        judge_text = judge_text.replace("</verduct>", "</verdict>")
        judge_text = judge_text.replace("</Verdict>", "</verdict>")
        judge_text = judge_text.replace("<Verdict>", "<verdict>")
        judge_text = judge_text.replace("<verdict>B</verdict>", "<verdict>[[B]]</verdict>")
        judge_text = judge_text.replace("<verdict> [B] </verdict>", "<verdict>[[B]]</verdict>")
        judge_text = judge_text.replace("Decision: [[A]]\n</verdict>", "<verdict>[[A]]</verdict>")

        
        # Extract content between <explanation> and </explanation>
        explanation_match = re.search(r'<explanation>(.*?)</explanation>', judge_text, re.DOTALL)
        explanation_text = explanation_match.group(1).strip() if explanation_match else None
        
        # Extract content between <verdict> and </verdict>
        verdict_match = re.search(r'<verdict>(.*?)</verdict>', judge_text, re.DOTALL)
        verdict_text = verdict_match.group(1).strip() if verdict_match else None

        try:
            assert verdict_text == "[[A]]" or verdict_text == "[[B]]", f"Unexpected verdict: {judge_text}"
        except:
            print("warning:", verdict_text)
            verdict_text = "[[C]]"
        
        if verdict_text == "[[A]]":
            if audio_a['mos'] > audio_b['mos']:
                count_correct += 1
                evaluation = "correct"
            else:
                evaluation = "incorrect"
        elif verdict_text == "[[B]]":
            if audio_a['mos'] < audio_b['mos']:
                count_correct += 1
                evaluation = "correct"
            else:
                evaluation = "incorrect"
        else:
                evaluation = "incorrect"
        count_all += 1
        
        predictions.append([evaluation, verdict_text, audio_a['mos'], audio_b['mos']])
    print("total:", count_all)
    print("accuracy: {:.2f}%".format(count_correct/count_all*100))
    return predictions

In [354]:
predictions_ab = read_output_file("../experiments/somos/ab_testing/diff15_prompt1_notext.txt")
# predictions_ab = read_output_file("../experiments/somos/ab_testing/diff15_prompt1_sanity.txt")

total: 148
accuracy: 57.43%


In [355]:
predictions_ab

[['correct', '[[A]]', 4.666666666666667, 2.4545454545454546],
 ['incorrect', '[[B]]', 4.090909090909091, 2.583333333333333],
 ['correct', '[[A]]', 4.333333333333333, 2.0],
 ['incorrect', '[[B]]', 4.333333333333333, 2.8181818181818183],
 ['incorrect', '[[B]]', 4.333333333333333, 2.6666666666666665],
 ['incorrect', '[[A]]', 2.0, 3.583333333333333],
 ['incorrect', '[[A]]', 2.1, 3.75],
 ['correct', '[[B]]', 2.0, 4.111111111111111],
 ['incorrect', '[[B]]', 4.0, 2.111111111111111],
 ['correct', '[[B]]', 1.875, 3.6666666666666665],
 ['correct', '[[A]]', 4.125, 2.583333333333333],
 ['incorrect', '[[B]]', 4.333333333333333, 2.7],
 ['incorrect', '[[B]]', 4.333333333333333, 2.7],
 ['incorrect', '[[B]]', 3.9, 2.3],
 ['correct', '[[B]]', 2.3, 3.875],
 ['correct', '[[A]]', 3.909090909090909, 2.111111111111111],
 ['incorrect', '[[B]]', 3.6, 1.1111111111111112],
 ['correct', '[[A]]', 3.6, 1.7692307692307692],
 ['correct', '[[A]]', 3.8, 1.1111111111111112],
 ['correct', '[[A]]', 3.8, 1.7692307692307692

In [356]:
predictions_ba = read_output_file("../experiments/somos/ab_testing/diff15_prompt1_BA_notext.txt", reverse=True)
# predictions_ba = read_output_file("../experiments/somos/ab_testing/diff15_prompt1_BA_sanity.txt", reverse=True)

total: 148
accuracy: 60.81%


In [357]:
predictions_ba

[['correct', '[[B]]', 2.4545454545454546, 4.666666666666667],
 ['correct', '[[B]]', 2.583333333333333, 4.090909090909091],
 ['correct', '[[B]]', 2.0, 4.333333333333333],
 ['incorrect', '[[A]]', 2.8181818181818183, 4.333333333333333],
 ['incorrect', '[[C]]', 2.6666666666666665, 4.333333333333333],
 ['correct', '[[A]]', 3.583333333333333, 2.0],
 ['correct', '[[A]]', 3.75, 2.1],
 ['correct', '[[A]]', 4.111111111111111, 2.0],
 ['correct', '[[B]]', 2.111111111111111, 4.0],
 ['incorrect', '[[B]]', 3.6666666666666665, 1.875],
 ['incorrect', '[[A]]', 2.583333333333333, 4.125],
 ['correct', '[[B]]', 2.7, 4.333333333333333],
 ['incorrect', '[[A]]', 2.7, 4.333333333333333],
 ['incorrect', '[[A]]', 2.3, 3.9],
 ['incorrect', '[[B]]', 3.875, 2.3],
 ['correct', '[[B]]', 2.111111111111111, 3.909090909090909],
 ['incorrect', '[[A]]', 1.1111111111111112, 3.6],
 ['incorrect', '[[A]]', 1.7692307692307692, 3.6],
 ['correct', '[[B]]', 1.1111111111111112, 3.8],
 ['incorrect', '[[A]]', 1.7692307692307692, 3.8

In [358]:
count_A, count_B, count = 0, 0, 0
for x in predictions_ab:
    count += 1
    if x[1] == "[[A]]":
        count_A += 1
    elif x[1] == "[[B]]":
        count_B += 1
    else:
        # raise Exception()
        pass
print("percentage_A: {:.2f}%".format(count_A/count*100))
print("percentage_B: {:.2f}%".format(count_B/count*100))

percentage_A: 52.70%
percentage_B: 40.54%


In [359]:
count_A, count_B, count = 0, 0, 0
for x in predictions_ba:
    count += 1
    if x[1] == "[[A]]":
        count_A += 1
    elif x[1] == "[[B]]":
        count_B += 1
    else:
        # raise Exception()
        pass
print("percentage_A: {:.2f}%".format(count_A/count*100))
print("percentage_B: {:.2f}%".format(count_B/count*100))

percentage_A: 52.03%
percentage_B: 44.59%


In [360]:
count_A, count_B, count = 0, 0, 0
for x in predictions_ab+predictions_ba:
    count += 1
    if x[1] == "[[A]]":
        count_A += 1
    elif x[1] == "[[B]]":
        count_B += 1
    else:
        # raise Exception()
        pass
print("percentage_A: {:.2f}%".format(count_A/count*100))
print("percentage_B: {:.2f}%".format(count_B/count*100))

percentage_A: 52.36%
percentage_B: 42.57%


In [361]:
abba_correct, abba_incorrect = 0, 0
for x in predictions_ab + predictions_ba:
    if x[0] == 'correct':
        abba_correct += 1
    elif x[0] == 'incorrect':
        abba_incorrect += 1
    else:
        raise Exception()
print("total:", abba_correct+abba_incorrect)
print("correct: {:.2f}%".format(abba_correct/(abba_correct+abba_incorrect)*100))
print("incorrect: {:.2f}%".format(abba_incorrect/(abba_correct+abba_incorrect)*100))

total: 296
correct: 59.12%
incorrect: 40.88%


In [362]:
predictions_ab = predictions_ab[:len(predictions_ba)]
consistent, bias_A, bias_B = 0, 0, 0
correct_given_consistent = 0
for ab, ba in zip(predictions_ab, predictions_ba):
    if ab[0] == ba[0]:
        consistent += 1
        if ab[0] == 'correct':
            correct_given_consistent += 1
    else:
        if ab[1] == "[[A]]" and ba[1] == "[[A]]":
            bias_A += 1
        elif ab[1] == "[[B]]" and ba[1] == "[[B]]":
            bias_B += 1
        else:
            # raise Exception("logic error")
            pass
# assert consistent + bias_A + bias_B == len(predictions_ab)
error = len(predictions_ab) - (consistent + bias_A + bias_B)

In [363]:
print("consistent: {:.2f}%".format(consistent/len(predictions_ab)*100))
print("bias_A:     {:.2f}%".format(bias_A/len(predictions_ab)*100))
print("bias_B:     {:.2f}%".format(bias_B/len(predictions_ab)*100))
print("error:      {:.2f}%".format(error/len(predictions_ab)*100))
print("P(correct|consistent): {:.2f}%".format(correct_given_consistent/consistent*100))

consistent: 53.38%
bias_A:     25.68%
bias_B:     16.89%
error:      4.05%
P(correct|consistent): 67.09%
